In [1]:
import pandas as pd
import numpy as np
import getpass
import json
import requests

1. Set up address screen call with `requests`
2. Parse out response of API response to csv columns
3. Load csv with addresses in first column
4. For each row in CSV, run Address Screening API
5. export results to CSV

`https://api.chainalysis.com/api/kyt/v1/users/{{userId}}/withdrawaladdresses`

In [2]:
##### ENTER YOUR API KEY
api_key = getpass.getpass("Enter your API Key:")

Enter your API Key: ································································


In [3]:
import requests
import json

url = "https://api.chainalysis.com/api/kyt/v1/users/User875/withdrawaladdresses"

payload = json.dumps([
  {
    "asset": "ETH",
    "address": "0x92193107FB10B3B372AB21cC90b5a4DBd67861d9"
  }
])

headers = {
  'token': '06094f6ef2909569d7533ecd8706cad35703709de49713299d6e74edb937b878',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


[{"asset":"ETH","address":"0x92193107FB10B3B372AB21cC90b5a4DBd67861d9","cluster":{"name":"Contact Chainalysis 0x9219","category":"stolen funds"},"rating":"lowRisk","customAddress":null,"chainalysisIdentification":null}]


In [4]:
pd.json_normalize(json.loads(response.text))


,asset,address,rating,customAddress,chainalysisIdentification,cluster.name,cluster.category
0,ETH,0x92193107FB10B3B372AB21cC90b5a4DBd67861d9,lowRisk,None,None,Contact Chainalysis 0x9219,stolen funds


In [77]:
input_csv = '~/Documents/Projects/batch-address-screen/sample_addresses.csv'
df = pd.read_csv(input_csv)

In [78]:
df.head()

,user_id,asset,address
0,1,eth,0x6acdfba02d390b97ac2b2d42a63e85293bcc160e
1,2,eth,0x7F19720A857F834887FC9A7bC0a0fBe7Fc7f8102
2,3,eth,0x92193107FB10B3B372AB21cC90b5a4DBd67861d9
3,4,eth,0xe1e8C9fa8F0cB41CcF9b92061b696EC247Fcc449
4,5,eth,0xC5Ca16CD564eA101bBF2Dba1346499289F142B07


### Iterate

In [79]:
data = []

for index, row in df.iterrows():
    user = row['user_id']
    asset = row['asset']
    address = row['address']
    
    # Load parsed inputs into a json object to be used in the payload of the API request
    newPayload = json.dumps([
        {"asset": asset,
        "address": address}])
    
    response = requests.request("POST", url, headers=headers, data=newPayload)
    data.append(json.loads(response.text).pop())
    
df_out = pd.DataFrame(pd.json_normalize(data))
df.drop("asset",axis=1,inplace=True)
df = df.merge(df_out,how="outer",on="address")

df.to_csv("Batch_Address_Screening.csv",encoding='utf8',index=False)

In [80]:
df

,user_id,address,asset,rating,customAddress,chainalysisIdentification,cluster.name,cluster.category,cluster
0,1,0x6acdfba02d390b97ac2b2d42a63e85293bcc160e,ETH,highRisk,None,None,OFAC SDN Chatex.com 2021-11-08,sanctions,NaN
1,2,0x7F19720A857F834887FC9A7bC0a0fBe7Fc7f8102,ETH,highRisk,None,None,OFAC SDN Secondeye Solution 2021-04-15,sanctions,NaN
2,3,0x92193107FB10B3B372AB21cC90b5a4DBd67861d9,ETH,lowRisk,None,None,Contact Chainalysis 0x9219,stolen funds,NaN
3,4,0xe1e8C9fa8F0cB41CcF9b92061b696EC247Fcc449,ETH,highRisk,None,None,MorphToken.com,high risk exchange,NaN
4,5,0xC5Ca16CD564eA101bBF2Dba1346499289F142B07,ETH,unknown,None,None,NaN,NaN,NaN


In [83]:
df.to_csv("Batch_Address_Screening.csv",encoding='utf8',index=False)